In [1]:
import sys
sys.path.append('../src')

import torch
import road as rd
import traffic_lights as tl
import junction as jn
import traffic_lights as tl
import network as nw
import initial_and_bc as ibc
import loading_json as load
import json
import bus

In [3]:
# Creating the roads
L = 25
N = 2
b = 4
init_fnc = lambda x : torch.ones_like(x) * 0.2
boundary_fnc = ibc.boundary_conditions(1, max_dens=1, densities=torch.tensor([0.2]), time_jumps=[], 
                                       in_speed=torch.tensor(50.0/3.6), L = L)
road_1 = rd.Road(b, L, N, [torch.tensor(40/3.6)], [], initial=init_fnc, boundary_fnc=boundary_fnc,
                 left_pos=(0, 3), right_pos=(2.9, 3), id="road_1", max_dens=1)
road_2 = rd.Road(b, L, N, [torch.tensor(40/3.6)], [], initial=init_fnc, boundary_fnc=None,
                 left_pos=(3.1, 3), right_pos=(6, 3), id = "road_2", max_dens=1)

road_3 = rd.Road(b, L, N, [torch.tensor(60/3.6)], [], initial=init_fnc, boundary_fnc=boundary_fnc,
                 left_pos=(0, 6), right_pos=(2.9, 6), id="road_3", max_dens=1)
road_4 = rd.Road(b, L, N, [torch.tensor(60/3.6)], [], initial=init_fnc, boundary_fnc=None,
                 left_pos=(3.1, 6), right_pos=(6, 6), id = "road_4", max_dens=1)

roads = [road_1, road_2, road_3, road_4]

# Creating the traffic light and the junction
junction_1 = jn.Junction([road_1, road_2], [0], [1], [[1.0]], [], [])
junction_2 = jn.Junction([road_3, road_4], [0], [1], [[1.0]], [], [])


# Creating the network
network = nw.RoadNetwork(roads, [junction_1, junction_2], T = 100)

# Creating the busses
bus_1_ids = ["road_1", "road_2"]
bus_1_stops = [("road_2", 50)]
bus_1_times = [0]
bus_1 = bus.Bus(bus_1_ids, bus_1_stops, bus_1_times, network, id = "bus_1")

bus_2_ids = ["road_3", "road_4"]
bus_2_stops = [("road_4", 50)]
bus_2_times = [0]
bus_2 = bus.Bus(bus_2_ids, bus_2_stops, bus_2_times, network, id = "bus_2")

bus_network = nw.RoadNetwork(roads, [junction_1, junction_2], T=100, roundabouts=[], busses=[bus_1, bus_2])


In [4]:
densities, _, bus_lengths, _ =  bus_network.solve_cons_law()

Bus bus_2 reached bus stop 0 at time 14.950634002685547, should wait for 29.64312171936035 seconds
Bus bus_1 reached bus stop 0 at time 35.9696159362793, should wait for 29.839807510375977 seconds


In [5]:
densities_loaded = load.convert_from_tensor(densities)
bus_lengths_loaded = load.convert_from_tensor(bus_lengths)

In [6]:
import json
with open("results/busses.json", 'w') as fd:
    fd.write(json.dumps([densities_loaded, bus_lengths_loaded]))

In [7]:
# Converting to mp4 and cropping video

In [8]:
import ffmpy


In [9]:
ff = ffmpy.FFmpeg(
    executable="C:\\Users\\torje\\Downloads\\ffmpeg-6.1-win-64\\ffmpeg.exe",
    inputs={'gifs/busses.gif': None},
    outputs={'videos/busses.mp4': None}
    )
ff.run()

(None, None)

In [1]:
from moviepy.editor import VideoFileClip,concatenate_videoclips
import moviepy.editor as mpy
from moviepy.video.fx.all import crop

In [4]:
# Cropping video
clip = VideoFileClip('videos/busses.mp4')
cropped_clip = crop(clip, width=750, height=400, x_center=400, y_center=300)
cropped_clip = cropped_clip.cutout(0, 0.1)
cropped_clip.write_videofile('videos/busses_cropped.mp4',codec="libx264")

Moviepy - Building video videos/busses_cropped.mp4.
Moviepy - Writing video videos/busses_cropped.mp4



Moviepy - Done !
Moviepy - video ready videos/busses_cropped.mp4
